<a href="https://colab.research.google.com/github/aSafarpoor/Dataset-Test/blob/main/MGTAB_small_Link_Prediction_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torch-geometric

In [15]:
# -*- coding: utf-8 -*-
"""Untitled40.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1tk3ns7BYA18FETuTkELImHk62hREKzqz
"""

# !pip install torch torch-geometric

import torch
import torch.nn.functional as F
from torch_geometric.data import Data as tgData
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from torch_geometric.utils import negative_sampling
from torch_geometric.transforms import RandomLinkSplit

try:
    from google.colab import output
    from tqdm.notebook import tqdm  # Colab/Jupyter notebook version
except ImportError:
    from tqdm import tqdm

from torch_geometric.nn import GATConv, MLP, SAGEConv, GCNConv
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import numpy as np
import random
from torch.nn import Linear

import copy

In [8]:
# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"device is {device}")
# device = torch.device('cpu')


# Load your data
y = torch.load("labels.pt")
x = torch.load("initial_embeddings.pt").to(device)
edge_index = torch.load("directed_edge_index.pt").to(device)

print(f"edges shape is {edge_index.shape}")
edge_index = edge_index.unique(dim=1)
print(f"edges shape is {edge_index.shape}")

x = F.normalize(x, p=2, dim=1)

# Generate train/val/test splits
split = RandomLinkSplit(is_undirected=False, split_labels=True,
                        num_val=0.1, num_test=0.1, add_negative_train_samples=True)

train_data, val_data, test_data = split(tgData(x=x, edge_index=edge_index))

device is cuda
edges shape is torch.Size([2, 720695])
edges shape is torch.Size([2, 425863])


In [12]:
class Net_Sage_as_Encoder_CosineSimilarity_as_Decoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.3):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(dropout)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.dropout(x)
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    @staticmethod
    def decode_topk(z, k):
        print(f"k is  {k}")
        z = F.normalize(z, p=2, dim=1)  # Normalize for cosine similarity
        topk_edges = []
        for i in tqdm(range(z.size(0)),desc="topkz"):
            sim = torch.matmul(z[i:i+1], z.t()).squeeze(0)
            sim[i] = -1  # Exclude self
            topk = torch.topk(sim, k=k).indices
            for j in topk:
                topk_edges.append([i, j.item()])
        print(len(topk_edges),z.size)
        return torch.tensor(topk_edges, dtype=torch.long).t()

model = Net_Sage_as_Encoder_CosineSimilarity_as_Decoder(x.size(1), 256, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()


# Train
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    pos_edge_index = train_data.pos_edge_label_index
    neg_edge_index = train_data.neg_edge_label_index

    edge_label_index = torch.cat([pos_edge_index, neg_edge_index], dim=1)
    edge_label = torch.cat([
        torch.ones(pos_edge_index.size(1), device=device),
        torch.zeros(neg_edge_index.size(1), device=device)
    ], dim=0)

    pred = model.decode(z, edge_label_index).view(-1)
    loss = criterion(pred, edge_label)
    loss.backward()
    optimizer.step()
    return loss.item()


# Evaluate
@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)

    edge_label_index = torch.cat([
        data.pos_edge_label_index,
        data.neg_edge_label_index,
    ], dim=1)

    edge_label = torch.cat([
        torch.ones(data.pos_edge_label_index.size(1), device=device),
        torch.zeros(data.neg_edge_label_index.size(1), device=device),
    ], dim=0)

    pred = model.decode(z, edge_label_index).view(-1).sigmoid()
    return roc_auc_score(edge_label.cpu(), pred.cpu())

# Training loop
best_model_state = None
best_val = final_test = 0
for epoch in tqdm(range(1, 1001),desc="training"):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val:
        best_val = val_auc
        final_test = test_auc
        best_model_state = copy.deepcopy(model.state_dict())
    if epoch%40==0:
        print(f'Epoch {epoch:03d} | Loss: {loss:.4f} | Val AUC: {val_auc:.4f} | Test AUC: {test_auc:.4f}')

print(f'Best Test AUC: {final_test:.4f}')

training:   0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 010 | Loss: 0.6184 | Val AUC: 0.8552 | Test AUC: 0.8537
Epoch 020 | Loss: 0.5454 | Val AUC: 0.8457 | Test AUC: 0.8469
Epoch 030 | Loss: 0.5364 | Val AUC: 0.8670 | Test AUC: 0.8678
Epoch 040 | Loss: 0.5314 | Val AUC: 0.8665 | Test AUC: 0.8678
Epoch 050 | Loss: 0.5239 | Val AUC: 0.8757 | Test AUC: 0.8768
Epoch 060 | Loss: 0.5162 | Val AUC: 0.8907 | Test AUC: 0.8913
Epoch 070 | Loss: 0.5082 | Val AUC: 0.9017 | Test AUC: 0.9020
Epoch 080 | Loss: 0.5020 | Val AUC: 0.9047 | Test AUC: 0.9053
Epoch 090 | Loss: 0.4965 | Val AUC: 0.9107 | Test AUC: 0.9114
Epoch 100 | Loss: 0.4935 | Val AUC: 0.9159 | Test AUC: 0.9166
Epoch 110 | Loss: 0.4898 | Val AUC: 0.9165 | Test AUC: 0.9174
Epoch 120 | Loss: 0.4883 | Val AUC: 0.9162 | Test AUC: 0.9170
Epoch 130 | Loss: 0.4843 | Val AUC: 0.9152 | Test AUC: 0.9159
Epoch 140 | Loss: 0.4825 | Val AUC: 0.9240 | Test AUC: 0.9245
Epoch 150 | Loss: 0.4808 | Val AUC: 0.9162 | Test AUC: 0.9167
Epoch 160 | Loss: 0.4798 | Val AUC: 0.9256 | Test AUC: 0.9258
Epoch 17

In [13]:
# Generate predicted edges for completion
z = model.encode(x, edge_index)
predicted_edges = model.decode_topk(z, k=20)

# Optionally: Combine original and predicted
# completed_edge_index = torch.cat([edge_index, predicted_edges], dim=1).unique(dim=1)
predicted_edges = predicted_edges.to(device)
print(predicted_edges.shape)
print(edge_index.shape)

completed_edge_index = torch.cat([edge_index, predicted_edges], dim=1).unique(dim=1)

# labels = torch.load("labels_bot.pt")

completed_edge_index = torch.unique(completed_edge_index, dim=1)

print(f"Original edge_index.shape       = {edge_index.shape}")
print(f"Completed edge_index.shape      = {completed_edge_index.shape}")
print(f"Newly added edges count         = {completed_edge_index.shape[1] - edge_index.shape[1]}")

k is  20


topkz:   0%|          | 0/10199 [00:00<?, ?it/s]

203980 <built-in method size of Tensor object at 0x78f6581d5d30>
torch.Size([2, 203980])
torch.Size([2, 425863])
Original edge_index.shape       = torch.Size([2, 425863])
Completed edge_index.shape      = torch.Size([2, 614725])
Newly added edges count         = 188862


In [14]:
torch.save(completed_edge_index, "completed_edge_index.pt")

In [16]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = GATConv(in_channels, 8, heads=8, dropout=0.6)
        self.conv2 = GATConv(64, out_channels, heads=1, concat=False, dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        return self.conv2(x, edge_index)

class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, 64)
        self.conv2 = GCNConv(64, out_channels)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        return self.conv2(x, edge_index)

class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, 64)
        self.conv2 = SAGEConv(64, out_channels)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        return self.conv2(x, edge_index)

class MLP(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.lin1 = Linear(in_channels, 64)
        self.lin2 = Linear(64, out_channels)

    def forward(self, x, edge_index=None):  # edge_index is unused, for API compatibility
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.6, training=self.training)
        return self.lin2(x)

In [17]:
def evaluate(model, x, edge_index, y, mask, num_labeled):
    model.eval()
    with torch.no_grad():
        logits = model(x, edge_index) if edge_index is not None else model(x)
        logits = logits[:num_labeled]  # restrict to labeled part
        probs = F.softmax(logits, dim=1)[:, 1]
        preds = logits.argmax(dim=1)
        mask = mask[:num_labeled]  # ensure mask matches y and logits
        acc = accuracy_score(y[mask].cpu(), preds[mask].cpu())
        auc = roc_auc_score(y[mask].cpu(), probs[mask].cpu())
        f1 = f1_score(y[mask].cpu(), preds[mask].cpu())
        return acc, auc, f1

def random_split(num_nodes, seed):
    np.random.seed(seed)
    perm = np.random.permutation(num_nodes)
    n_train = int(0.6 * num_nodes)
    n_val = int(0.2 * num_nodes)
    train_idx = perm[:n_train]
    val_idx = perm[n_train:n_train + n_val]
    test_idx = perm[n_train + n_val:]
    return train_idx, val_idx, test_idx

In [24]:

number_of_experiments = 3

results = {"GAT_orig": [],
           "GAT_comp": [],
           "GCN_orig": [],
           "GCN_comp": [],
           "GraphSAGE_orig": [],
           "GraphSAGE_comp": [],
           "MLP": []}

def set_all_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

# Only use the labeled portion
num_labeled = y.size(0)
for exp in range(number_of_experiments):
    set_all_seeds(exp)

    # Split only the labeled indices
    train_idx, val_idx, test_idx = random_split(num_labeled, exp)

    # Create masks for the full x
    train_mask = torch.zeros(x.size(0), dtype=torch.bool)
    val_mask = torch.zeros(x.size(0), dtype=torch.bool)
    test_mask = torch.zeros(x.size(0), dtype=torch.bool)

    train_mask[train_idx] = True
    val_mask[val_idx] = True
    test_mask[test_idx] = True

    # Print class ratios
    for name, mask in zip(["Train", "Val", "Test"], [train_mask, val_mask, test_mask]):
        mask_labeled = mask[:num_labeled]
        b = (y[mask_labeled] == 0).sum().item()
        s = (y[mask_labeled] == 1).sum().item()
        total = mask_labeled.sum().item()
        print(f"[Exp {exp}] {name} - Benign: {b/total:.2f}, Sybil: {s/total:.2f}")

    # === GAT on original edge_index ===
    model = GAT(x.size(1), 2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
    for epoch in tqdm(range(200), desc="original gat"):
        model.train()
        optimizer.zero_grad()
        out = model(x.to(device), edge_index.to(device))
        loss = F.cross_entropy(out[train_mask], y[train_mask[:num_labeled]].to(device))
        loss.backward()
        optimizer.step()
    results["GAT_orig"].append(evaluate(model, x.to(device), edge_index.to(device), y.to(device), test_mask, num_labeled))


    # === GAT on completed_edge_index ===
    model = GAT(x.size(1), 2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
    for epoch in tqdm(range(200), desc="complete gat"):
        model.train()
        optimizer.zero_grad()
        out = model(x.to(device), completed_edge_index.to(device))
        loss = F.cross_entropy(out[train_mask], y[train_mask[:num_labeled]].to(device))
        loss.backward()
        optimizer.step()
    results["GAT_comp"].append(evaluate(model, x.to(device), completed_edge_index.to(device), y.to(device), test_mask, num_labeled))

    # === GCN on original edge_index ===
    model = GCN(x.size(1), 2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
    for epoch in tqdm(range(200), desc="original GCN"):
        model.train()
        optimizer.zero_grad()
        out = model(x.to(device), edge_index.to(device))
        loss = F.cross_entropy(out[train_mask], y[train_mask[:num_labeled]].to(device))
        loss.backward()
        optimizer.step()
    results["GCN_orig"].append(evaluate(model, x.to(device), edge_index.to(device), y.to(device), test_mask, num_labeled))


    # === GCN on completed_edge_index ===
    model = GCN(x.size(1), 2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
    for epoch in tqdm(range(200), desc="complete GCN"):
        model.train()
        optimizer.zero_grad()
        out = model(x.to(device), completed_edge_index.to(device))
        loss = F.cross_entropy(out[train_mask], y[train_mask[:num_labeled]].to(device))
        loss.backward()
        optimizer.step()
    results["GCN_comp"].append(evaluate(model, x.to(device), completed_edge_index.to(device), y.to(device), test_mask, num_labeled))

    # === GraphSAGE on original edge_index ===
    model = GraphSAGE(x.size(1), 2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
    for epoch in tqdm(range(200), desc="original GraphSAGE"):
        model.train()
        optimizer.zero_grad()
        out = model(x.to(device), edge_index.to(device))
        loss = F.cross_entropy(out[train_mask], y[train_mask[:num_labeled]].to(device))
        loss.backward()
        optimizer.step()
    results["GraphSAGE_orig"].append(evaluate(model, x.to(device), edge_index.to(device), y.to(device), test_mask, num_labeled))


    # === GraphSAGE on completed_edge_index ===
    model = GraphSAGE(x.size(1), 2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
    for epoch in tqdm(range(200), desc="complete GraphSAGE"):
        model.train()
        optimizer.zero_grad()
        out = model(x.to(device), completed_edge_index.to(device))
        loss = F.cross_entropy(out[train_mask], y[train_mask[:num_labeled]].to(device))
        loss.backward()
        optimizer.step()
    results["GraphSAGE_comp"].append(evaluate(model, x.to(device), completed_edge_index.to(device), y.to(device), test_mask, num_labeled))

    # === MLP ===
    model = MLP(x.size(1), 2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
    for epoch in tqdm(range(200), desc="mlp"):
        model.train()
        optimizer.zero_grad()
        out = model(x.to(device))
        loss = F.cross_entropy(out[train_mask], y[train_mask[:num_labeled]].to(device))
        loss.backward()
        optimizer.step()
    results["MLP"].append(evaluate(model, x.to(device), None, y.to(device), test_mask, num_labeled))


[Exp 0] Train - Benign: 0.73, Sybil: 0.27
[Exp 0] Val - Benign: 0.73, Sybil: 0.27
[Exp 0] Test - Benign: 0.73, Sybil: 0.27


original gat:   0%|          | 0/200 [00:00<?, ?it/s]

complete gat:   0%|          | 0/200 [00:00<?, ?it/s]

original GCN:   0%|          | 0/200 [00:00<?, ?it/s]

complete GCN:   0%|          | 0/200 [00:00<?, ?it/s]

...

original GCN:   0%|          | 0/200 [00:00<?, ?it/s]

complete GCN:   0%|          | 0/200 [00:00<?, ?it/s]

original GraphSAGE:   0%|          | 0/200 [00:00<?, ?it/s]

complete GraphSAGE:   0%|          | 0/200 [00:00<?, ?it/s]

mlp:   0%|          | 0/200 [00:00<?, ?it/s]

In [25]:
# === Print Final Results ===
for name, scores in results.items():
    arr = np.array(scores)
    print(f"{name}:")
    print(f"  ACC: {arr[:,0].mean():.4f} ± {arr[:,0].std():.4f}")
    print(f"  AUC: {arr[:,1].mean():.4f} ± {arr[:,1].std():.4f}")
    print(f"  F1 : {arr[:,2].mean():.4f} ± {arr[:,2].std():.4f}")

GAT_orig:
  ACC: 0.7475 ± 0.0030
  AUC: 0.7982 ± 0.0010
  F1 : 0.1155 ± 0.0040
GAT_comp:
  ACC: 0.7808 ± 0.0034
  AUC: 0.8322 ± 0.0024
  F1 : 0.4116 ± 0.0550
GCN_orig:
  ACC: 0.7785 ± 0.0062
  AUC: 0.7962 ± 0.0032
  F1 : 0.3703 ± 0.0197
GCN_comp:
  ACC: 0.7865 ± 0.0095
  AUC: 0.8285 ± 0.0024
  F1 : 0.4974 ± 0.0373
GraphSAGE_orig:
  ACC: 0.8811 ± 0.0073
  AUC: 0.9411 ± 0.0056
  F1 : 0.7691 ± 0.0180
GraphSAGE_comp:
  ACC: 0.8623 ± 0.0028
  AUC: 0.9388 ± 0.0057
  F1 : 0.7023 ± 0.0379
MLP:
  ACC: 0.8584 ± 0.0080
  AUC: 0.9254 ± 0.0067
  F1 : 0.7114 ± 0.0100
